## Web-scraping Covid-19 data
#### Anas Puthawala

Libraries used: pandas, requests, BeautifulSoup, and datetime.
The date of the pull is a few lines down

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as soup
from datetime import datetime as dt

In [2]:
#Columns wanted: Country, Total Cases, New Cases, New Deaths, TotalRecovered, ActiveCases, CriticalCases, Totalcase1m, 
# totaltests, totaltest1m

### Pulling website HTML source code using requests library

In [3]:
URL = 'https://www.worldometers.info/coronavirus/#countries%27'

In [4]:
date_execution = dt.now()
html_source = requests.get(URL).text

In [5]:
# mm/dd/YY H:M:S
dt_string = date_execution.strftime("%m/%d/%Y %H:%M:%S")
print("date and time of html pull =", dt_string)

#DATE / TIME OF PULL BELOW

date and time of html pull = 12/22/2021 20:40:45


In [6]:
# html_source

### Utilizing BeautifulSoup to scrape HTML source code for required columns

In [7]:
soup = soup(html_source)

In [8]:
tableid = 'main_table_countries_today'
table_main = soup.find('table', id=tableid)
# table_main

In [9]:
table_data = table_main.tbody.find_all('tr', style=["", "background-color:#EAF7D5", "background-color:#F0F0F0"])

In [10]:
# table_data

### Making a dictionary to convert later into pandas dataframe

In [11]:
d = {}
t = []
for i in table_data:
    cols = i.find_all('td')
    cols_info = [col.text.strip() for col in cols][1:15]
    d[cols_info[0]] = cols_info[1:]

In [12]:
df = pd.DataFrame(d).transpose()

In [13]:
df.index.rename('Country', inplace=True)
column_names=['TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'New Recovered', 'ActiveCases', 'CriticalCases', 'Totcase1M', 'Totdeath1M', 'TotalTests', 'Tottest1M', 'Population']
df.columns = column_names
df

,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,New Recovered,ActiveCases,CriticalCases,Totcase1M,Totdeath1M,TotalTests,Tottest1M,Population
Country,,,,,,,,,,,,,
USA,"52,506,200",,"832,939",,"40,908,146",,"10,765,115","16,197","157,270","2,495","795,121,064","2,381,595","333,860,677"
India,"34,763,305",,"478,468",,"34,201,966",,"82,871","8,944","24,831",342,"667,356,171","476,691","1,399,976,270"
Brazil,"22,222,928",,"618,128",,"21,414,318",,"190,482","8,318","103,466","2,878","63,776,166","296,932","214,784,057"
UK,"11,647,473",,"147,573",,"9,922,480",,"1,577,420",849,"170,257","2,157","392,136,238","5,732,054","68,411,120"
Russia,"10,292,983",,"300,269",,"9,097,521",,"895,193","2,300","70,487","2,056","236,300,000","1,618,198","146,026,630"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Samoa,3,,,,3,,0,,15,,,,"200,355"
Saint Helena,2,,,,2,,0,,328,,,,"6,104"
Micronesia,1,,,,1,,0,,9,,,,"116,798"


Dropping unwanted columns: 'New Recovered', 'Population'

In [14]:
unwanted_cols = ['New Recovered', 'Population']

In [15]:
df = df.drop(unwanted_cols, axis=1)

In [16]:
df

,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Totcase1M,Totdeath1M,TotalTests,Tottest1M
Country,,,,,,,,,,,
USA,"52,506,200",,"832,939",,"40,908,146","10,765,115","16,197","157,270","2,495","795,121,064","2,381,595"
India,"34,763,305",,"478,468",,"34,201,966","82,871","8,944","24,831",342,"667,356,171","476,691"
Brazil,"22,222,928",,"618,128",,"21,414,318","190,482","8,318","103,466","2,878","63,776,166","296,932"
UK,"11,647,473",,"147,573",,"9,922,480","1,577,420",849,"170,257","2,157","392,136,238","5,732,054"
Russia,"10,292,983",,"300,269",,"9,097,521","895,193","2,300","70,487","2,056","236,300,000","1,618,198"
...,...,...,...,...,...,...,...,...,...,...,...
Samoa,3,,,,3,0,,15,,,
Saint Helena,2,,,,2,0,,328,,,
Micronesia,1,,,,1,0,,9,,,


### Removing unwanted characters / Converting numbers to ints

In [17]:
# From inspection we can see that the 'NewCases' and 'NewDeaths' column is the only column that has unwanted characters ('+').
# The list of unwanted characters is “/”, “\n”, “|”, “+”

df['NewCases'] = df['NewCases'].str.replace('+','')
df['NewDeaths'] = df['NewDeaths'].str.replace('+','')

#Replace the commas as well since excel files will initially pick it up as a string and need to get converted to number afterwards
df.replace(',','', regex=True, inplace=True) #Will get rid of the ',' so excel can pick it up as numbers easily


df = df.apply(pd.to_numeric, errors='coerce')

<ipython-input-17-4c431fd71451>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['NewCases'] = df['NewCases'].str.replace('+','')
<ipython-input-17-4c431fd71451>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['NewDeaths'] = df['NewDeaths'].str.replace('+','')


In [18]:
df.dtypes

TotalCases          int64
NewCases          float64
TotalDeaths       float64
NewDeaths         float64
TotalRecovered    float64
ActiveCases       float64
CriticalCases     float64
Totcase1M         float64
Totdeath1M        float64
TotalTests        float64
Tottest1M         float64
dtype: object

### Outputting it as a excel file

In [19]:
file_name = 'Covid_19_webscrape.xlsx'
df.to_excel(file_name)